In [8]:
#!pip install mysql
#!pip install mysql-connector-python-rf
#import warnings
#warnings.filterwarnings("ignore")

You are using pip version 9.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
from flask import Flask
from flask import url_for, render_template, request, redirect
import mysql.connector as mysql



In [49]:
session = {}
session['username'] = None
global_data = None

In [50]:
app = Flask(__name__)

In [51]:
con = mysql.connect(user = 'root', password = 'password', 
                    host = '127.0.0.1', database = 'db_proj', #database = 'task_manager'
                   auth_plugin='mysql_native_password')
cur = con.cursor(dictionary=True)

In [52]:
@app.route('/home')
def index():
    if session['username'] == None:
        return render_template('home.html')
    else:
        tasks = []
        importance = []
        statuses = [] 
        cur.execute("SELECT tasks_list.task_name as 'task', importance_list.importance_name as 'importance',\
status_list.status_name as 'status' FROM user_tasks, users_list, tasks_list, status_list, db_proj.importance_list \
WHERE users_list.username = '" + session['username'] + "' and users_list.id_user = user_tasks.id_user \
and tasks_list.id_task = user_tasks.id_task and status_list.id_status = user_tasks.id_status \
and importance_list.id_importance = user_tasks.id_importance")
        result = cur.fetchall()
        #with open('log.txt', 'a', encoding='utf-8') as r:
        #    r.write(str(result)) 
        for res in result:
            tasks.append(res['task'])
            importance.append(res['importance'])
            statuses.append(res['status'])
        return render_template('home_logged_in.html', tasks=tasks, importance=importance, statuses=statuses)

In [53]:
@app.route('/signup', methods=['GET', 'POST'])
def sign_up():
    if request.method == "POST":
        details = request.form
        global_data = details['username']
        username = details['username']
        password = details['password']
        cur.execute("INSERT INTO users_list (username, password) VALUES (%s, %s)", (username, password))
        con.commit()
        cur.close()
        return redirect('login')
    return render_template('sign_up.html')

In [54]:
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == "POST":
        details = request.form
        username = details['username']
        password = details['password']
        cur.execute("SELECT * FROM users_list")
        users = cur.fetchall()
        user_exists = 0
        for i in users:
            if i['username'] == username:
                user_exists = 1
                if i['password'] == password:
                    session['username'] = username
                    return redirect('home')
                else:
                    return ('Неправильный пароль.')
        if user_exists == 1:
            return redirect ('home')
        else:
            return ('Такого пользователя пока нет.')
    return render_template('login.html')

In [55]:
@app.route('/new_task', methods=['GET', 'POST'])
def new_task(): 
    if request.method == "POST":
        details = request.form
        global_data = details['tasks']
        cur.execute("SELECT id_user FROM users_list WHERE username = '" + session['username'] + "'")
        res = cur.fetchone()
        id_user = res['id_user']

        cur.execute("SELECT id_task FROM tasks_list WHERE task_name = '" + details['tasks'] + "'")
        res = cur.fetchone()
        id_task = res['id_task']
        
        cur.execute("SELECT id_status FROM status_list WHERE status_name = '" + details['status'] + "'")
        res = cur.fetchone()
        id_status = res['id_status']
        
        cur.execute("SELECT id_importance FROM importance_list WHERE importance_name = '" + details['importance'] + "'")
        res = cur.fetchone()
        id_importance = res['id_importance']
        
        data = (id_user, id_task, id_status, id_importance)
        cur.execute("INSERT INTO user_tasks (id_user, id_task, id_status, id_importance) VALUES (%s, %s, %s, %s)", data)
        con.commit()
        return ('Задача успешно создана!')
    tasks = []
    importance = []
    status = []
    cur.execute("SELECT task_name FROM tasks_list")
    res = cur.fetchall()
    for i in res:
        tasks.append(i['task_name'])
    cur.execute("SELECT importance_name FROM importance_list")
    res = cur.fetchall()
    for i in res:
        importance.append(i['importance_name'])
    cur.execute("SELECT status_name FROM status_list")
    res = cur.fetchall()
    for i in res:
        status.append(i['status_name'])
    return render_template('new_task.html', tasks = tasks, status = status, importance = importance)

In [ ]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Dec/2019 20:52:10] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2019 20:52:13] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2019 20:52:14] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [18/Dec/2019 20:52:14] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2019 20:52:20] "GET /new_task HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2019 20:52:25] "POST /new_task HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2019 20:54:13] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2019 20:59:31] "GET /new_task HTTP/1.1" 200 -


In [26]:
this = (1, 3, 2, 1)
cur.execute("INSERT INTO user_tasks (id_user, id_task, id_status, id_importance) VALUES (%s, %s, %s, %s)", this)
con.commit()

In [124]:
session['username'] = 'fishowl'
cur.execute("SELECT id_user FROM users_list WHERE username = '" + session['username'] + "'")

In [152]:
cur.execute("SELECT * FROM user_tasks")
res = cur.fetchall()
res

[{'id_importance': 3,
  'id_status': 2,
  'id_task': 5,
  'id_user': 1,
  'id_user_task': 1},
 {'id_importance': 3,
  'id_status': 3,
  'id_task': 5,
  'id_user': 2,
  'id_user_task': 2},
 {'id_importance': 3,
  'id_status': 1,
  'id_task': 2,
  'id_user': 1,
  'id_user_task': 3},
 {'id_importance': 1,
  'id_status': 2,
  'id_task': 3,
  'id_user': 1,
  'id_user_task': 9},
 {'id_importance': 3,
  'id_status': 2,
  'id_task': 3,
  'id_user': 1,
  'id_user_task': 10},
 {'id_importance': 1,
  'id_status': 1,
  'id_task': 5,
  'id_user': 1,
  'id_user_task': 11}]

In [24]:
res['id_user']

TypeError: list indices must be integers or slices, not str

In [131]:
tasks = []
importance = []
statuses = []

cur.execute("SELECT tasks_list.task_name as \'task\', importance_list.importance_name as \'importance\',\
status_list.status_name as \'status\' FROM user_tasks, users_list, tasks_list, status_list, db_proj.importance_list \
WHERE users_list.username = '" + test_username + "' and users_list.id_user = user_tasks.id_user \
and tasks_list.id_task = user_tasks.id_task and status_list.id_status = user_tasks.id_status \
and importance_list.id_importance = user_tasks.id_importance")
result = cur.fetchall()
with open('test.txt', 'w', encoding='utf-8') as r:
    r.write(str(result))
for res in result:
    tasks.append(res['task'])
    importance.append(res['importance'])
    statuses.append(res['status'])
statuses

['в процессе']

In [31]:
cur.execute("DELETE FROM user_tasks WHERE id_user_task > 3")
con.commit()

In [7]:
con.commit()

In [113]:
global_data